In [ ]:
import os
import openai
import pandas as pd
import time

# OpenAI API 키 설정
openai.api_key = os.getenv('OPENAI_API_KEY')

# 데이터 폴더 경로
data_folder_path = "./data/"

# 폴더 내 모든 CSV 파일 가져오기
file_list = [f for f in os.listdir(data_folder_path) if f.endswith('.csv')]

# 파일마다 감성 분석 수행
for file_name in file_list:
    review_file_path = os.path.join(data_folder_path, file_name)
    reviews_df = pd.read_csv(review_file_path)

    # 감성 분석 결과를 저장할 리스트
    sentiments = []

    # 리뷰마다 감성 분석 수행
    for review_text in reviews_df['Review']:  # 컬럼명이 'Review'라고 가정
        try:
            # 최신 ChatCompletion 모델을 사용한 감성 분석
            response = openai.ChatCompletion.create(
                model="gpt-3.5-turbo-1106",
                messages=[
                    {"role": "system", "content": "당신은 감성 분석을 전문으로 하는 도움말 어시스턴트입니다. 문맥이 없는 단어(예: '좋아요', '별로예요')만 있는 경우, 단어의 감성에 따라 '긍정적' 또는 '부정적'으로 응답하세요."},
                    {"role": "user", "content": f"다음 리뷰의 감성을 분석하여 '긍정적' 또는 '부정적' 형식으로만 응답해 주세요: '{review_text}'"}
                ],
                max_tokens=10,  # 응답 길이를 줄입니다.
                temperature=0.2
            )
            
            # API 응답에서 감성 분석 결과 추출
            response_text = response.choices[0].message['content'].strip()
            if "긍정적" in response_text:
                sentiment = "긍정적"
            elif "부정적" in response_text:
                sentiment = "부정적"
            else:
                sentiment = "알 수 없음"
            
            sentiments.append(sentiment)
            
        except Exception as e:
            print(f"리뷰 처리 중 오류 발생: {e}")
            sentiments.append("오류")
            time.sleep(1)  # 오류 발생 시 잠시 대기
        
        # 요청 빈도 제어
        time.sleep(0.5)  # 각 요청 사이에 대기 시간 추가

    # 결과를 데이터프레임에 추가
    reviews_df['sentiment'] = sentiments

    # 원본 파일 이름에 "_분석" 추가하여 저장
    base_name = os.path.splitext(file_name)[0]
    output_path = f"./data/{base_name}_분석.csv"
    reviews_df.to_csv(output_path, index=False)
    print(f"감성 분석 완료 및 결과 저장: {output_path}")
